In [2]:
!pip install selenium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [104]:
import pandas as pd 
import os
import re 
import numpy as np


from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='/Users/alexaj/anaconda3/bin/chromedriver')

In [105]:
url = "https://degreeworks.banner.marist.edu/dashboard/dashboard"
driver.get(url)

In [106]:
#driver.implicitly_wait(5)
contFrame = driver.find_element_by_css_selector('html > frameset > frame:nth-child(4)')
driver.switch_to.frame(contFrame)
#frBC = driver.find_element_by_xpath('/html/frameset')
bodyFrame = driver.find_element_by_name('frBody')
driver.switch_to.frame(bodyFrame)

In [46]:
html = driver.page_source
soup = BeautifulSoup(html, "html5lib")

In [85]:
#taking majors titles, minor titles, pathway
headTitles = soup.find_all("td", attrs={"class": "BlockHeadTitle"})
headTitle = []
for head in headTitles:
    headTitle.append(head.text)

majorArray = [x for x in headTitle if 'Major' in x]
minorArray = [x for x in headTitle if 'Minor' in x]
degreeName = [x for x in headTitle if 'Degree' in x]
pwayName = [x for x in headTitle if 'Pathway' in x][0]

In [151]:
#code for required courses 
reqStrings = soup.find_all("td", attrs={"class": "RuleLabelTitleNeeded"})

reqCourses = []
for reqString in reqStrings:
    reqCourses.append(reqString.text)

coreReqs = [x for x in reqCourses if '3 credits' in x]
#coreReqs
coreReqDf = pd.DataFrame(coreReqs)
coreReqDf.columns = ['Core Requirement']

,Core Requirement
0,3 credits Ethics/Applied Ethics/Religious Studies


In [107]:
pathwayClasses = driver.find_element_by_css_selector('#frmAudit > table:nth-child(28) > tbody > tr > td > table > tbody > tr:nth-child(5) > td.RuleLabelData > table')
pathwaySoup = BeautifulSoup(pathwayClasses.get_attribute('innerHTML'), "html5lib")

pathwayTable = pathwaySoup.find_all('table')[0]
pathwayDf = pd.read_html(str(pathwayTable))[0]
pathwayDf.columns = ['Course No', 'Course Title', 'Grade', 'Credits', 'Semester']
pathwayDf

# pathwayFileJSON = 'SerenityPathway.JSON'
# pathwayDf.to_json(pathwayFileJSON)
# pathwayFileCSV = 'SerenityPathway.csv'
# pathwayDf.to_json(pathwayFileCSV, index=False)

,Course No,Course Title,Grade,Credits,Semester
0,MATH 130L,INTRO-STATIST I,A,3,Spring 2016
1,MATH 241L,CALCULUS I,B+,4,Fall 2016
2,ECON 104L,PRINC MACROECONOM,B,3,Fall 2017
3,CMPT 120L,INTRO TO PROGRAMMING,B+,4,Spring 2016


In [58]:
#Code for scraped data from Student View information
studentTable = soup.find("table", attrs={"class": "Inner"})
studentDataTag = studentTable.findAll('td')

studentData = []

for data in studentDataTag:
    #Below one line extracts data 
    studentData.append(data.text.strip())

#studentData
#studentTitle = studentData[::2]
#studentD = studentData[1::2]
#studentTitle
keyVal = dict(zip(studentData[::2], studentData[1::2]))
studentView = pd.DataFrame.from_dict(keyVal, orient='index')
studentView

,0
Student,"Javellana, Alexa"
Level,Undergraduate
ID,20068940
Degree,B.S.
Classification,Junior
College,Computer Science & Mathematics
Advisor,"Cannistra, Robert M"
Major,Computer Science
Overall GPA,3.476
Minor,Math


In [49]:
#transfer this dataframe into a CSV
filename = 'SerenityStudentInfo.csv'
studentView.to_json(filename)

In [32]:
semYear = datetime.date.today().year
semMonth = datetime.date.today().month
semSsn = ""

if (semMonth >= 1 and semMonth <= 5):
  semSsn = "Spring"
elif (semMonth >= 6 and semMonth <= 8):
  semSsn = "Summer"
elif (semMonth >= 9 and semMonth <= 12):
  semSsn = "Fall"
elif (semMonth == 1):
  semSsn = "Winter"

semSsn + " " + str(semYear)

'Fall 2018'

In [9]:
currClassHTML = soup.find_all("table", attrs={"class": "xBlocks"})[1]

In [27]:
currClassHTML = soup.find_all("table", attrs={"class": "xBlocks"})[1]

#classNames

className = currClassHTML.find_all("td", attrs={"class": "SectionCourseTitle"})
classNames = []

for classes in className:
    classNames.append(classes.text)
    
#classNumbers

classNo = currClassHTML.find_all("td", attrs={"class": "ClassesAppliedClasses"})
classNos = []

for no in classNo:
    no = no.text.replace(u'\xa0', u' ')
    classNos.append(no)

#creditsEach

noCredits = currClassHTML.find_all("td", attrs={"class": "SectionCourseCredits"})
curCredits = []

for noCreds in noCredits:
    curCredits.append(noCreds.text)
    
# curCredits

In [28]:
currClassDataStack = np.vstack((classNos, classNames, curCredits)).T
currClassDataStack = currClassDataStack.tolist()

currClassInfoDf = pd.DataFrame(currClassDataStack,columns=['Course No', 'Course Title', 'Credit Value'])
currClassInfoDf

,Course No,Course Title,Credit Value
0,BUS 120N,FINANCIAL LITERACY,1
1,CMPT 331L,THEORY OF PROGRAM LANG,3
2,CMPT 422N,COMPUTER ORG & ARCH,4
3,CMPT 424N,OPERATING SYSTEMS,4
4,CMPT 475N,CS PROJECT,3
5,CMPT 476N,CS PROJECT II,1
6,PHED 127N,TENNIS,1


In [102]:
#This code parses the data on DegreeWorks by td class, BlockHeadSubd
#Code parses credits into total and completed arrays

creditTitles = []
creditTitles = degreeName + majorArray + minorArray

subData = soup.find_all("td", attrs={"class": "BlockHeadSubData"})
#dataNP = array of subData that is not parsed, but converted from obj to str
subDataNP = []
for subD in subData:
    subDataNP.append(subD.text)
    #objToText

#dataP = array subData that is/will be parsed
subDataP = []
for subDataN in subDataNP:
    string = subDataN.split("\xa0 ",1)[1]
    if string:
        subDataP.append(string)
#subDataP
    
credits = []

for subData in subDataP:
    if not any(s in subData for s in ('-', '.')):
        credits.append(subData)

totalCredits = []
completedCredits = []
i = 0
while i < len(credits):
    if i % 2 == 0:
        totalCredits.append(credits[i])
        i += 1
    else:
        completedCredits.append(credits[i])
        i += 1
        
creditsProgress = np.vstack((creditTitles, completedCredits, totalCredits)).T
creditsProgress = creditsProgress.tolist()
progressDf = pd.DataFrame(creditsProgress,columns=['Title', 'Credits Completed', 'Total Needed'])
progressDf

,Title,Credits Completed,Total Needed
0,Degree in Bachelor of Science,104,120
1,Major in Computer Science,64,67
2,Minor in Math,8,18


In [17]:
ppmDatas = soup.find_all("span", attrs={"class": "item_count"})
ppmNames = soup.find_all("span", attrs={"class": "item_name"})

In [201]:
ppmInfo = np.vstack((ppmCountry, ppm)).T
ppmInfo = ppmInfo.tolist()

In [202]:
df = pd.DataFrame(ppmInfo,columns=['By country', 'Page views per minute'])

In [204]:
df

,By country,Page views per minute
0,GB,0.0176
1,US,0.011
2,IE,0.00347
3,DE,0.00288
4,AT,0.00278
5,BR,0.00198
6,SK,0.00169
7,HU,0.00139
8,RU,0.00129
9,VE,0.000794


In [205]:
filename = '0816CountryView.csv'
df.to_csv(filename, index=False)
#os.rename("/Users/alexaj/LastWeekConsolidation/" + filename, "/Users/alexaj/LastWeekConsolidation/AJAXData/" + filename)